In [1]:
import os
os.chdir('..')
from inputGetter import get_input
from functools import reduce
from operator import ixor

content = get_input(2017, 14)[:-1]

In [2]:
def knot_hash(lengths_ls, rounds_cnt):
    NUM = 256
    ls = [i for i in range(NUM)]
    position = 0
    
    for round in range(rounds_cnt):
        for skip_sz, length in enumerate(lengths_ls):
            if length > NUM:
                continue
            
            # Reverse the list segment
            start = position
            end = position + length - 1
            while start <= end:
                ls[start % NUM], ls[end % NUM] = ls[end % NUM], ls[start % NUM]
                start += 1
                end -= 1
            
            # Get to new position
            actual_skip = skip_sz + len(lengths_ls) * round  # The skip is retained through each round, so I have to add previous rounds in too
            position += length + actual_skip    
    
    return ls

In [3]:
'''This is an extension of the second part of the day 10 within the same year. 
Go check that out to understand this knot hash algorithm'''
def get_hash(row):
    global content
    data = content + f"-{row}"
    data = [ord(val) for val in data]
    data.extend([17, 31, 73, 47, 23])

    # Get sparse hash
    ls = knot_hash(data, 64)

    # Make dense hash, then get the whole 128-bits representation (total of all 16 bits segments)
    res = []
    for i in range(0, len(ls), 16):
        xor_val = reduce(ixor, ls[i: i+16], 0)  # XOR the whole segment
        res.extend([int(bit) for bit in f'{xor_val:08b}'])
        
    return res

In [4]:
# Clear a region
def bfs(disk_content, row, col):
    disk_content[row][col] = 0
    if row > 0 and disk_content[row-1][col] == 1:
        bfs(disk_content, row-1, col)
    if row < 127 and disk_content[row+1][col] == 1:
        bfs(disk_content, row+1, col)
    if col > 0 and disk_content[row][col-1] == 1:
        bfs(disk_content, row, col-1)
    if col < 127 and disk_content[row][col+1] == 1:
        bfs(disk_content, row, col+1)

disk_content = [get_hash(row) for row in range(128)]
used_squares = sum(sum(row) for row in disk_content)
regions_count = 0
for row in range(128):
    for col in range(128):
        if disk_content[row][col] == 1:
            regions_count += disk_content[row][col]
            bfs(disk_content, row, col)

print(f'Part A: {used_squares}')
print(f'Part B: {regions_count}')

Part A: 8140
Part B: 1182
